# Calculating Nearest Mrt Station To Bus Stop

In [1]:
%run lta_datamall_data.ipynb

In [2]:
# bus stops geodataframe
bus_stops_gdf = geopandas.GeoDataFrame(bus_stops, 
                                       geometry = geopandas.points_from_xy(bus_stops.Longitude, bus_stops.Latitude),
                                       crs = "EPSG:4326")
bus_stops_gdf

,BusStopCode,RoadName,Description,Latitude,Longitude,geometry
0,01012,Victoria St,Hotel Grand Pacific,1.296848,103.852536,POINT (103.85254 1.29685)
1,01013,Victoria St,St. Joseph's Ch,1.297710,103.853225,POINT (103.85322 1.29771)
2,01019,Victoria St,Bras Basah Cplx,1.296990,103.853022,POINT (103.85302 1.29699)
3,01029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414,POINT (103.85441 1.29667)
4,01039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491,POINT (103.85549 1.29821)
...,...,...,...,...,...,...
5135,99139,Changi Village Rd,Blk 5,1.388195,103.987234,POINT (103.98723 1.38819)
5136,99161,Nicoll Dr,Aft Changi Beach CP 3,1.390262,103.992957,POINT (103.99296 1.39026)
5137,99171,Nicoll Dr,Changi Beach CP 2,1.391128,103.991021,POINT (103.99102 1.39113)
5138,99181,Telok Paku Rd,Bef S'pore Aviation Ac,1.387754,103.988503,POINT (103.98850 1.38775)


In [3]:
# bus routes geodataframe
bus_routes_gdf = geopandas.GeoDataFrame(bus_route_names, 
                                       geometry = geopandas.points_from_xy(bus_route_names.Longitude, bus_route_names.Latitude),
                                       crs = "EPSG:4326")
bus_routes_gdf

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus,RoadName,Description,Latitude,Longitude,geometry
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300,Tampines Ctrl 1,Tampines Int,1.354076,103.943391,POINT (103.94339 1.35408)
1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302,Tampines Ave 5,Opp Our Tampines Hub,1.352962,103.941652,POINT (103.94165 1.35296)
2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304,Tampines Ave 5,Blk 147,1.348753,103.942086,POINT (103.94209 1.34875)
3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308,Simei Ave,Changi General Hosp,1.340055,103.948381,POINT (103.94838 1.34005)
4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309,Simei Ave,Opp Blk 3012,1.337371,103.950673,POINT (103.95067 1.33737)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25543,9B,SBST,1,25,95091,9.5,0741,0817,-,-,-,-,Nicoll Dr,SAF Ferry Ter,1.387214,103.999714,POINT (103.99971 1.38721)
25544,9B,SBST,1,26,95131,9.7,0742,0818,-,-,-,-,Airline Rd,Police Pass Off,1.385252,103.999834,POINT (103.99983 1.38525)
25545,9B,SBST,1,27,95141,10.2,0744,0820,-,-,-,-,Airline Rd,Aft Cargo Bldg D,1.381386,103.998191,POINT (103.99819 1.38139)
25546,9B,SBST,1,28,95061,10.6,0745,0821,-,-,-,-,Airline Rd,Airline Hse,1.377552,103.996539,POINT (103.99654 1.37755)


In [ ]:
# train station exits geodataframe
train_station_exits_gdf = geopandas.read_file("./TrainStationExit/Train_Station_Exit_Layer.shp")
train_station_exits_gdf

In [ ]:
bus_routes_gdf_3857 = bus_routes_gdf.to_crs(3857)
train_station_exits_gdf_3857 = train_station_exits_gdf.to_crs(3857)

nearest_train_station_output = []
for i, row in tqdm(bus_routes_gdf_3857.iterrows()):

    # Compute the distance from each bus stop to all train station exits
    distances = train_station_exits_gdf_3857.distance(row['geometry'])

    nearest_train_station_output.append({
        'Bus Service': row['ServiceNo'],
        'Route Direction': row['Direction'],
        'Bus Stop Code': row['BusStopCode'],
        'Description': row['Description'],
        'Nearest Train Station': train_station_exits_gdf_3857.loc[int(distances.argmin()), 'stn_name'],
        'Distance': float(distances.min())
    })

#finding nearest stations
nearest_train_stations = pd.DataFrame(nearest_train_station_output)
nearest_train_stations

In [ ]:
bus_stops_with_mrt = nearest_train_stations[nearest_train_stations['Distance'] < 150]
number_of_mrt = bus_stops_with_mrt.groupby(['Bus Service', 'Route Direction']).size().reset_index(name = 'Count')
number_of_mrt.sort_values('Count', ascending = False)

# Bus Routes Data

In [ ]:
from shapely.geometry import LineString

# combine bus routes
bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()
bus_routes_combined.columns = ['ServiceNo', 'Direction', 'geometry']
bus_routes_combined

# MRT Stations Data

In [ ]:
# mrt stations geodataframe
import os
os.environ['OGR_GEOMETRY_ACCEPT_UNCLOSED_RING'] = 'NO'

mrt_stations_gdf = geopandas.read_file('./TrainStation_Jul2024/RapidTransitSystemStation.shp')
mrt_stations_gdf_3857 = mrt_stations_gdf.to_crs(3857)
mrt_stations_gdf_3857

In [ ]:
mrt_stations_gdf2 = geopandas.read_file('./mrt_stations/mrt_stations.shp')

mrt_stations_gdf2["lat_lng"] = geopandas.points_from_xy(mrt_stations_gdf2["LATITUDE"], mrt_stations_gdf2["LONGITUDE"])
mrt_stations_gdf2["line"] = mrt_stations_gdf2["CODE"].str.slice(0, 2) 

mrt_lines_code = ["NS", "EW", "CC", "NE", "TE", "DT", "CG"]

mrt_gdf_wo_lrt = mrt_stations_gdf2[mrt_stations_gdf2["line"].isin(mrt_lines_code)].reset_index()
mrt_gdf_wo_lrt = mrt_gdf_wo_lrt.drop("index", axis = 1)

mrt_gdf_wo_lrt["STN_NAM_DE"] = mrt_gdf_wo_lrt["BUILDING"].str.split('(').str[0].str.strip()

mrt_gdf_wo_lrt2 = mrt_gdf_wo_lrt[["CODE", "STATION_NA", "STN_NAM_DE", "LINE", "line", "COLOR", "geometry"]]

mrt_gdf_wo_lrt2["Order"] = pd.to_numeric(mrt_gdf_wo_lrt2["CODE"].str.slice(2, None))

In [ ]:
new_brown_line = ['TANJONG RHU MRT STATION','KATONG PARK MRT STATION', 'TANJONG KATONG MRT STATION', 
                  'MARINE PARADE MRT STATION', 'MARINE TERRACE MRT STATION', 'SIGLAP MRT STATION', 'BAYSHORE MRT STATION']

In [ ]:
n = len(new_brown_line)
add_tel_stations = []
for i in range(n):
    no = i + 23
    station_name = new_brown_line[i]
    add_tel_stations.append({
        "CODE": "TE" + str(no),
        "STATION_NA": station_name,
        "STN_NAM_DE": station_name,
        "LINE": "Thomson-East Coast Line",
        "line": "TE",
        "COLOR": "Brown",
        "Order": no
    })

new_tel_stations = pd.DataFrame(add_tel_stations)

mrt_gdf_wo_lrt2 = pd.concat([mrt_gdf_wo_lrt2, new_tel_stations], axis = 0)
mrt_gdf_wo_lrt2

In [ ]:
mrt_stations_3857_2 = mrt_gdf_wo_lrt2.merge(mrt_stations_gdf_3857, how = "left", on = "STN_NAM_DE")

mrt_stations_3857_2

### Thomson-East Coast Line

In [ ]:
TEL = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "TE"]["STATION_NA"].tolist()

# filter stations from mrt stations geodataframe
TEL_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "TE"]

TEL_stations_gdf["order"] = pd.to_numeric(TEL_stations_gdf["CODE"].str.slice(2, None))

# order stations based on list
TEL_stations_gdf = TEL_stations_gdf.sort_values('order')
TEL_stations_gdf

#### Route Visualisation for Thomson-East Coast Line

In [ ]:
import folium
import geopandas
from shapely.geometry import mapping
from shapely.geometry import LineString

# ordered list of station coordinates
TEL_stations_gdf['coordinates'] = TEL_stations_gdf['geometry_y'].apply(lambda geom: geom.centroid)
TEL_stations_gdf

# create a linestring for the  route
TEL_route = LineString(TEL_stations_gdf['coordinates'])

# convert from LineString to geodataframe
TEL_route_gdf = geopandas.GeoDataFrame(geometry = [TEL_route], crs = "EPSG:3857")
TEL_route_gdf_4326 = TEL_route_gdf.to_crs(4326)

# create base map
TEL_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add TEL route layer
folium.GeoJson(
    data = mapping(TEL_route_gdf_4326),
    name = 'TEL',
    style_function = lambda x: {
        'color': '#9D5B25',
        'weight': 4
    }
).add_to(TEL_map)

TEL_map

In [ ]:
# calculate the overlapping distance of each bus route and the TEL
buffer_distance = 150
buffered_TEL = TEL_route.buffer(buffer_distance)

overlap_distance_output_TEL = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_TEL.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_TEL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'Thomson-East Coast Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_TEL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'Thomson-East Coast Line',
            'Intersection': overlap
            })

overlap_distance_TEL = geopandas.GeoDataFrame(overlap_distance_output_TEL)

# filter bus routes with overlap
bus_routes_overlap_TEL = overlap_distance_TEL[overlap_distance_TEL['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_TEL

In [ ]:
bus_routes_overlap = bus_routes_overlap_TEL
bus_routes_overlap

### Downtown Line 

In [ ]:
DTL = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "DT"]["STATION_NA"].tolist()

# filter stations from mrt stations geodataframe
DTL_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "DT"]

DTL_stations_gdf["order"] = pd.to_numeric(DTL_stations_gdf["CODE"].str.slice(2, None))

# order stations based on list
DTL_stations_gdf = DTL_stations_gdf.sort_values('order')
DTL_stations_gdf

#### Route Visualisation for Downtown Line

In [ ]:
# ordered list of  station coordinates
DTL_stations_gdf['coordinates'] = DTL_stations_gdf['geometry_y'].apply(lambda geom: geom.centroid)
DTL_stations_gdf

# create a linestring for the route
DTL_route = LineString(DTL_stations_gdf['coordinates'])

# convert from LineString to geodataframe
DTL_route_gdf = geopandas.GeoDataFrame(geometry = [DTL_route], crs = "EPSG:3857")
DTL_route_gdf_4326 = DTL_route_gdf.to_crs(4326)

# create base map
DTL_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add DTL route layer
folium.GeoJson(
    data = mapping(DTL_route_gdf_4326),
    name = 'DTL',
    style_function = lambda x: {
        'color': '#005ec4',
        'weight': 4
    }
).add_to(DTL_map)

DTL_map

In [ ]:
# calculate the overlapping distance of each bus route and the blue line
buffered_DTL = DTL_route.buffer(buffer_distance)

overlap_distance_output_DTL = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_DTL.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_DTL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'Downtown Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_DTL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'Downtown Line',
            'Intersection': overlap
            })

overlap_distance_DTL = pd.DataFrame(overlap_distance_output_DTL)
overlap_distance_DTL

# filter bus routes with overlap with DTL
bus_routes_overlap_DTL = overlap_distance_DTL[overlap_distance_DTL['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_DTL

In [ ]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_DTL], axis = 0)
bus_routes_overlap

### North-East Line 

In [ ]:
NEL = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "NE"]["STATION_NA"].tolist()

# filter stations from mrt stations geodataframe
NEL_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "NE"]

NEL_stations_gdf["order"] = pd.to_numeric(NEL_stations_gdf["CODE"].str.slice(2, None))

# order stations based on list
NEL_stations_gdf = NEL_stations_gdf.sort_values('order')
NEL_stations_gdf

#### Route Visualisation for North-East Line

In [ ]:
# ordered list of purple line station coordinates
NEL_stations_gdf['coordinates'] = NEL_stations_gdf['geometry_y'].apply(lambda geom: geom.centroid)
NEL_stations_gdf

# create a linestring for the route
NEL_route = LineString(NEL_stations_gdf['coordinates'])

# convert from LineString to geodataframe
NEL_route_gdf = geopandas.GeoDataFrame(geometry = [NEL_route], crs = "EPSG:3857")
NEL_route_gdf_4326 = NEL_route_gdf.to_crs(4326)

# create base map
NEL_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add NEL route layer
folium.GeoJson(
    data = mapping(NEL_route_gdf_4326),
    name = 'NEL',
    style_function = lambda x: {
        'color': '#9900aa',
        'weight': 4
    }
).add_to(NEL_map)

NEL_map

In [ ]:
# calculate the overlapping distance of each bus route and the line
buffered_NEL = NEL_route.buffer(buffer_distance)

overlap_distance_output_NEL = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_NEL.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_NEL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'North-East Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_NEL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'North-East Line',
            'Intersection': overlap
            })

overlap_distance_NEL = pd.DataFrame(overlap_distance_output_NEL)
overlap_distance_NEL

# filter bus routes with overlap with NEL line
bus_routes_overlap_NEL = overlap_distance_NEL[overlap_distance_NEL['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_NEL

In [ ]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_NEL], axis = 0)
bus_routes_overlap

### North-South Line 

In [ ]:
NSL = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "NS"]["STATION_NA"].tolist()

# filter red line stations from mrt stations geodataframe
NSL_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "NS"]

NSL_stations_gdf["order"] = pd.to_numeric(NSL_stations_gdf["CODE"].str.slice(2, None))

# order red line stations based on list
NSL_stations_gdf = NSL_stations_gdf.sort_values('order')
NSL_stations_gdf

#### Route Visualisation for North-South Line

In [ ]:
# ordered list of red line station coordinates
NSL_stations_gdf['coordinates'] = NSL_stations_gdf['geometry_y'].apply(lambda geom: geom.centroid)
NSL_stations_gdf


# create a linestring for the red line route
NSL_route = LineString(NSL_stations_gdf['coordinates'])
NSL_route

# convert from LineString to geodataframe
NSL_route_gdf = geopandas.GeoDataFrame(geometry = [NSL_route], crs = "EPSG:3857")
NSL_route_gdf_4326 = NSL_route_gdf.to_crs(4326)

# create base map
NSL_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add NSL route layer
folium.GeoJson(
    data = mapping(NSL_route_gdf_4326),
    name = 'NEL',
    style_function = lambda x: {
        'color': '#d42e12',
        'weight': 4
    }
).add_to(NSL_map)

NSL_map

In [ ]:
buffered_NSL = NSL_route.buffer(buffer_distance)

overlap_distance_output_NSL = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_NSL.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_NSL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'North-South Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_NSL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'North-South Line',
            'Intersection': overlap
            })

overlap_distance_NSL = pd.DataFrame(overlap_distance_output_NSL)
overlap_distance_NSL

bus_routes_overlap_NSL = overlap_distance_NSL[overlap_distance_NSL['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_NSL

In [ ]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_NSL], axis = 0)
bus_routes_overlap

### East-West Line 

In [ ]:
EWL = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "EW"]["STATION_NA"].tolist()
EWL_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "EW"]

EWL_stations_gdf["order"] = pd.to_numeric(EWL_stations_gdf["CODE"].str.slice(2, None))

# order stations based on list
EWL_stations_gdf = EWL_stations_gdf.sort_values('order')
EWL_stations_gdf

#### Route Visualisation for East-West Line

In [5]:
EWL_stations_gdf = geopandas.GeoDataFrame(EWL_stations_gdf, geometry='geometry_y')

EWL_stations_gdf['coordinates'] = EWL_stations_gdf['geometry_y'].apply(lambda geom: geom.centroid)
EWL_stations_gdf

# create a linestring for the route
EWL_route = LineString(EWL_stations_gdf['coordinates'])
EWL_route

# convert from LineString to geodataframe
EWL_route_gdf = geopandas.GeoDataFrame(geometry = [EWL_route], crs = "EPSG:3857")
EWL_route_gdf_4326 = EWL_route_gdf.to_crs(4326)

# create base map
EWL_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add EWL route layer
folium.GeoJson(
    data = mapping(EWL_route_gdf_4326),
    name = 'NEL',
    style_function = lambda x: {
        'color': '#009645',
        'weight': 4
    }
).add_to(EWL_map)

EWL_map

NameError: name 'EWL_stations_gdf' is not defined

In [ ]:
buffered_EWL = EWL_route.buffer(buffer_distance)

overlap_distance_output_EWL = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_EWL.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_EWL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'East-West Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_EWL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'East-West Line',
            'Intersection': overlap
            })

overlap_distance_EWL = pd.DataFrame(overlap_distance_output_EWL)
overlap_distance_EWL

bus_routes_overlap_EWL = overlap_distance_EWL[overlap_distance_EWL['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_EWL

In [ ]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_EWL], axis = 0)
bus_routes_overlap

### Circle Line

In [ ]:
CCL = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "CC"]["STATION_NA"].tolist()

CCL_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "CC"]

CCL_stations_gdf["order"] = pd.to_numeric(CCL_stations_gdf["CODE"].str.slice(2, None))

# order circle stations based on list
CCL_stations_gdf = CCL_stations_gdf.sort_values('order')
CCL_stations_gdf

#### Route Visualisation for Circle Line

In [ ]:
CCL_stations_gdf = geopandas.GeoDataFrame(CCL_stations_gdf, geometry='geometry_y')

CCL_stations_gdf['coordinates'] =CCL_stations_gdf['geometry_y'].apply(lambda geom: geom.centroid)
CCL_stations_gdf

# create a linestring for the circle line route
CCL_route = LineString(CCL_stations_gdf['coordinates'])
CCL_route

# convert from LineString to geodataframe
CCL_route_gdf = geopandas.GeoDataFrame(geometry = [CCL_route], crs = "EPSG:3857")
CCL_route_gdf_4326 = CCL_route_gdf.to_crs(4326)

# create base map
CCL_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add CCL route layer
folium.GeoJson(
    data = mapping(CCL_route_gdf_4326),
    name = 'CCL',
    style_function = lambda x: {
        'color': '#fa9e0d',
        'weight': 4
    }
).add_to(CCL_map)

CCL_map

In [ ]:
buffered_CCL = CCL_route.buffer(buffer_distance)

overlap_distance_output_CCL = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_CCL.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_CCL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'Circle Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_CCL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'Circle Line',
            'Intersection': overlap
            })

overlap_distance_CCL = pd.DataFrame(overlap_distance_output_CCL)
overlap_distance_CCL

bus_routes_overlap_CCL = overlap_distance_CCL[overlap_distance_CCL['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_CCL

In [ ]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_CCL], axis = 0)
bus_routes_overlap

# Bus Stops in Overlap

In [ ]:
# find the bus stops that belong to each intersection between the bus routes and the MRT lines
bus_routes_overlap_gdf_3857 = geopandas.GeoDataFrame(bus_routes_overlap,
                                                     geometry = bus_routes_overlap['Intersection'],
                                                     crs = "EPSG:3857")

overlap_count_output = []

for i, row in tqdm(bus_routes_overlap_gdf_3857.iterrows()):
    bus_no = row['Bus Service']
    direction = row['Direction']
    bus = bus_routes_gdf_3857[(bus_routes_gdf_3857['ServiceNo'] == bus_no) & (bus_routes_gdf_3857['Direction'] == direction)]
    for j, stop in bus.iterrows():
        if row['Intersection'].contains(stop.geometry):
            overlap_count_output.append({
                'Bus Service': bus_no,
                'Direction': direction,
                'Bus Stop Code': stop['BusStopCode'],
                'Description': stop['Description'],
                'MRT line': row['MRT line'],
                'geometry': stop.geometry
            })

overlap_bus_stops = geopandas.GeoDataFrame(overlap_count_output, crs = "EPSG:3857")


In [ ]:
# get the number of bus stops in the intersection between each bus route mrt line
overlap_count = overlap_bus_stops.groupby(['Bus Service', 'Direction', 'MRT line']).size().reset_index(name = 'Count').sort_values('Count', ascending = False)

bus_routes_overlap = bus_routes_overlap.merge(overlap_count, on = ['Bus Service', 'Direction', 'MRT line'], how = 'outer')
bus_routes_overlap = bus_routes_overlap[bus_routes_overlap['Count'].notna()]
bus_routes_overlap['Count'] = bus_routes_overlap['Count'].astype(int)

In [ ]:
# top 10 routes with significant overlap
bus_routes_overlap.sort_values(['Overlap Distance'], ascending = False).head(10)